<a href="https://colab.research.google.com/github/andronikmk/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

# Method 1

In [0]:
# TODO - Words, words, mere words, no matter from the heart.

In [0]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 47kB/s 
     |████████████████████████████████| 3.8MB 42.9MB/s 
     |████████████████████████████████| 450kB 42.8MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
# import libraries
import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

print(tf.__version__)

2.0.0


In [0]:
# import zipped text file
from zipfile import ZipFile

file_name = "/content/text.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
data_files = os.listdir('/content/text')

In [0]:
# Read in Data

data = []

for file in data_files:
    if file[-3:] == 'txt':
        with open(f'./text/{file}', 'r', encoding='utf-8') as f:
            data.append(f.read())

In [0]:
# Encode Data as Chars

# Gather all text 
# Why? 1. See all possible characters 2. For training / splitting later
text = " ".join(data)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [0]:
len(chars)

107

In [0]:
# Create the sequence data

maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1114623


In [0]:
sequences[0]

[47,
 72,
 8,
 46,
 51,
 64,
 65,
 13,
 40,
 19,
 33,
 35,
 40,
 65,
 39,
 74,
 65,
 46,
 60,
 11,
 31,
 19,
 104,
 66,
 65,
 19,
 16,
 51,
 58,
 43,
 41,
 65,
 40,
 65,
 19,
 102,
 51,
 46,
 49,
 31]

In [0]:
# Create x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [0]:
x.shape

(1114623, 40, 107)

In [0]:
y.shape

(1114623, 107)

In [0]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)), dropout=.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               120832    
_________________________________________________________________
dense (Dense)                (None, 107)               13803     
Total params: 134,635
Trainable params: 134,635
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    # Takes predictions and translates them into a character.
    # Into a position of a chracter.
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
# fit the model

model.fit(x, y,
          batch_size=1024,
          epochs=10,
          callbacks=[print_callback])

Train on 1114623 samples
Epoch 1/10
1114112/1114623 [============================>.] - ETA: 0s - loss: 2.7499
----- Generating text after Epoch: 0
----- Generating with seed: "but two shirts
  out with me, and I mean"
but two shirts
  out with me, and I mean,
      s loand moly thy ae calltht foel
 hethders ap; fapenpe noty; oo hor! UNATPLERU. Hlor Ice. mithe louahf'n.
s  Ms nir, Aghin dorsos, yod ar Ont eiee,  yo watn ord gheis.
Dart Phawcee fou to nou dUs not frans arertoTho; to  is iw shekoTh serings wont lo:e pfro,

     Er slo kin horlas? I I  lothat yf maan; o d seit entin  I the thut
or womesl tes, ant,  entsen hyy
Hut t yon.

 CURARE. Hacgfit
1114623/1114623 [==============================] - 1549s 1ms/sample - loss: 2.7498
Epoch 2/10
1114112/1114623 [============================>.] - ETA: 0s - loss: 2.3622
----- Generating text after Epoch: 1
----- Generating with seed: "ich, unrevers'd, stands in effectual for"
ich, unrevers'd, stands in effectual for

merseebll niw, of lid g

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN

# Method 2

In [3]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 77kB/s 
     |████████████████████████████████| 3.8MB 36.6MB/s 
     |████████████████████████████████| 450kB 60.8MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [1]:
import numpy as np
import random
import sys
import tensorflow as tf

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
print(tf.__version__)

2.0.0


In [2]:
### Words, words, mere words, no matter from the heart.
url = 'https://www.gutenberg.org/files/100/100-0.txt'
filepath = tf.keras.utils.get_file(fname='shakespeare.txt',
                                   origin=url)

5783552/5777367 [==============================] - 1s 0us/step


In [0]:
# open full text via file path
full_text = open(filepath, 'r', encoding='utf-8-sig').read()

In [0]:
# sonnets in list form
sonnets = full_text[2771:101122]

In [0]:
def preprocess(text):
    text = ' '.join(text.split())
    return text

In [0]:
sonnets = preprocess(sonnets)

In [9]:
# Identify unique characters.
chars = sorted(set(sonnets))
' '.join(chars)

'  ! ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; ? A B C D E F G H I J K L M N O P R S T U V W Y a b c d e f g h i j k l m n o p q r s t u v w x y z ‘ ’'

In [0]:
# Create mapping of unique chars to integers and vice versa
char_to_index = {c:i for i, c in enumerate(chars)}
index_to_char = {i:c for i, c in enumerate(chars)}

In [11]:
n_chars = len(sonnets)
n_vocab = len(chars)
print ('Total Characters:', n_chars)
print ('Unique Characters:', n_vocab)

Total Characters: 94192
Unique Characters: 71


In [12]:
# Generate the sequence data.

maxlen = 40
steps = 3

subsequences = [] # X 
next_chars = [] # Y

for i in range(0, len(sonnets) - maxlen, steps):
    subsequences.append(sonnets[i : i + maxlen])
    next_chars.append(sonnets[i + maxlen])

print ('Number of subsequences:', len(subsequences))

Number of subsequences: 31384


In [14]:
print("sub 0: ", subsequences[0])
print("sub 1: ", subsequences[1]) 

sub 0:  THE SONNETS 1 From fairest creatures we 
sub 1:   SONNETS 1 From fairest creatures we des


In [0]:
# Specify x & y.
x = np.zeros((len(subsequences), maxlen, len(chars)),
             dtype=np.bool)
y = np.zeros((len(subsequences), len(chars)), 
             dtype=np.bool)

for i, subsequence in enumerate(subsequences):
    for t, char in enumerate(subsequence):
        x[i, t, char_to_index[char]] = 1
        
    y[i, char_to_index[next_chars[i]]] = 1

In [16]:
# x shape
x.shape

(31384, 40, 71)

In [17]:
# y shape
y.shape

(31384, 71)

In [0]:
# Build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array.
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    """
    Function invoked at end of each epoch. Prints generated text.
    """
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(sonnets) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        subsequence = sonnets[start_index: start_index + maxlen]
        generated += subsequence
        print('----- Generating with seed: "' + subsequence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(subsequence):
                x_pred[0, t, char_to_index[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index_to_char[next_index]

            subsequence = subsequence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [21]:
# fit model
model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Train on 31384 samples
Epoch 1/5
31360/31384 [============================>.] - ETA: 0s - loss: 3.0021
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "earing sight, Serving with looks his sac"
earing sight, Serving with looks his sace ce  ie  ee  oe  oe  ee  e e  oe  ie  ae  o e  o  he  he  oe  eh  eh oe  e ee  on  e  e e de t i e e e  oe  ae  e  ee  he e  ho  the  ee  ee th e e  he  oe  ee  ee  oe  t  ee  he  ae  ee  ei ie e e  oe  en  e  he re  e  e e ie  oe  te  e ee  ee  tee  ee  he e  e e e e re  oe  oe e  he  he  he  e i ie  ie  ee  ee  o  oe  he  ee he  he  e  ee  oe  iee  ee  he  he  to  e e ee e ee  ee   oe e  ee e e
----- diversity: 0.5
----- Generating with seed: "earing sight, Serving with looks his sac"
earing sight, Serving with looks his sac sie e riei cee  ehe  diey  reei ahe nd ieo  te tey tolr e te re bh a  gy ce ie ader hreat feo se ae hi sooe d e haee  oee  wor  ar  o eo  ol bhe tan e h e tben r fr  me in   od me ma lee s cAs f